# <span style='color:SteelBlue'>P8 - Réalisez un dashboard et assurez une veille technique</span>

Image?

# <span style='background:red; color:black'>a MAJ</span>

# <span style='background:white; color:black'>Sommaire</span>

Importations des librairies<br>

Paramètres d'affichage<br>

Fonctions <br>

Importation des donnees <br>

**Etape 1 : Description des donnees**
- _1/ etc..._


Conclusion

# <span style='background:blue'>Introduction</span>

Veille technique...
___
**Missions**<br>
1/ ...


___
Ce notebook presente ...

___

**Source des datasets**<br>
a completer...

# <span style='background:grey'>Importations des librairies</span>

In [ ]:
# Librairies generales
import os.path
import sys
import time
from datetime import datetime

# Librairies data science
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Affichage plot dans le notebook
%matplotlib inline

# Versions
print("Version des librairies utilisees :")
print("Python        : " + sys.version)
print("Matplotlib    : " + mpl.__version__)
print("NumPy         : " + np.version.full_version)
print("Pandas        : " + pd.__version__)
print("Seaborn       : " + sns.__version__)

# Afficher heure lancement
maintenant = datetime.now().isoformat()
print("\nCode lance le : " + maintenant)

# Enregistrer l'heure de debut
start_time = time.time()

# Activer la verification PEP8
print("")
%reload_ext pycodestyle_magic
%pycodestyle_on

: 

# <span style='background:grey'>Paramètres d'affichage</span>

In [ ]:
# nombre ligne, col et chiffre apres la virgule
pd.set_option("display.max_row", 150)
pd.set_option("display.max_column", 150)
pd.set_option("display.precision", 2)

# <span style='background:grey'>Fonctions</span>


# <span style='background:grey'>Importation des donnees</span>

Importation des fichiers CSV avec Pandas

In [ ]:
# Definir le chemin
chemin = "./data/source/"

# Obtenir une liste de tous les fichiers dans le repertoire
fichiers = os.listdir(chemin)

# Initialiser un dictionnaire vide et print de la liste des fichiers importe
data_brut = {}
print("===> Liste des fichiers importes: <====\n")

# POUR chaque fichier
for fichier in fichiers:

    # Traiter uniquement les fichiers .csv
    if fichier.endswith(".csv"):
        print(fichier)

        # Lire le fichier csv et le stocker dans le dictionnaire
        data_brut[fichier] = pd.read_csv(
            chemin + fichier,
            sep=",",
            encoding="utf-8"
        )

# <span style='background:blue'>Etape 1 : Description des données</span>

## <span style='background:green'>1/ Nom</span>

### <span style='background:black'>a/ Sous-nom</span>

<div class="alert alert-block alert-success"><b>Bilan: </b> <br>
ici